In [1]:
import numpy as np
import pandas as pd
from numpy.random import randn

import matplotlib.pyplot as plt
import seaborn as sns
import dataprep as dp
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression, LogisticRegression, RANSACRegressor, Ridge, Lasso, ElasticNet

import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, MaxAbsScaler
from sklearn.model_selection import cross_val_score,KFold
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression, RANSACRegressor, ElasticNet
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

In [10]:
train_df = pd.read_csv('train.csv')
train_df.drop(['id'], axis=1, inplace=True)
column_names = list(train_df.columns.values.tolist())

In [11]:
train_df.drop(index=[934, 1035], inplace=True)

In [12]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1457 entries, 0 to 1458
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   hour                    1457 non-null   int64  
 1   hour_bef_temperature    1457 non-null   float64
 2   hour_bef_precipitation  1457 non-null   float64
 3   hour_bef_windspeed      1450 non-null   float64
 4   hour_bef_humidity       1457 non-null   float64
 5   hour_bef_visibility     1457 non-null   float64
 6   hour_bef_ozone          1383 non-null   float64
 7   hour_bef_pm10           1369 non-null   float64
 8   hour_bef_pm2.5          1342 non-null   float64
 9   count                   1457 non-null   float64
dtypes: float64(9), int64(1)
memory usage: 125.2 KB


In [13]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

train = train_df.copy()

train = IterativeImputer(random_state=2021).fit_transform(train)

train_imp = pd.DataFrame(train)
train_imp.columns = column_names

train_imp.isna().sum()

hour                      0
hour_bef_temperature      0
hour_bef_precipitation    0
hour_bef_windspeed        0
hour_bef_humidity         0
hour_bef_visibility       0
hour_bef_ozone            0
hour_bef_pm10             0
hour_bef_pm2.5            0
count                     0
dtype: int64

In [14]:
from sklearn.ensemble import IsolationForest 
model=IsolationForest(n_estimators=100, max_samples='auto', n_jobs=-1, max_features=2, contamination=0.01, random_state=2021)
model.fit(train_imp.to_numpy())

IsolationForest(contamination=0.01, max_features=2, n_jobs=-1,
                random_state=2021)

In [15]:
score = model.decision_function(train_imp.to_numpy())
anomaly = model.predict(train_imp.to_numpy())
train_imp['scores']= score
train_imp['anomaly']= anomaly
anomaly_data = train_imp.loc[train_imp['anomaly']==-1] # 이상값은 -1으로 나타낸다.
anomaly_data

,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count,scores,anomaly
4,18.0,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.000000,431.0,-0.034030,-1
118,20.0,22.7,0.0,3.1,41.0,732.0,0.100,102.0,60.000000,249.0,-0.006103,-1
154,0.0,11.4,1.0,2.8,84.0,350.0,0.012,69.0,59.000000,17.0,-0.000209,-1
233,14.0,23.1,0.0,3.6,44.0,620.0,0.080,237.0,62.000000,91.0,-0.003583,-1
363,3.0,11.6,1.0,0.0,90.0,284.0,0.010,68.0,57.000000,5.0,-0.015720,-1
501,21.0,21.3,0.0,0.8,47.0,715.0,0.092,100.0,56.000000,229.0,-0.005032,-1
636,6.0,14.7,1.0,0.1,93.0,153.0,0.020,59.0,31.000000,3.0,-0.000443,-1
641,16.0,30.0,0.0,3.2,16.0,1183.0,0.100,70.0,38.000000,304.0,-0.011722,-1
675,18.0,26.7,0.0,4.2,32.0,913.0,0.095,86.0,47.000000,286.0,-0.003426,-1
983,2.0,17.9,1.0,0.1,91.0,383.0,0.004,29.0,26.000000,22.0,-0.004355,-1


In [19]:
outlier_index=train_imp.index[train_imp['anomaly']==-1].tolist()
for i in outlier_index:
    train_imp=train_imp.drop(index=i,axis=0)
    
train_imp=train_imp.drop(train_imp[['anomaly']],axis=1)
train_imp=train_imp.drop(train_imp[['scores']],axis=1)

In [21]:
X=train_imp.drop('count',axis=1)
y=train_imp['count']
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.3, random_state=2021)

In [ ]:
from sklearn.ensemble import StackingRegressor

best_reg = ExtraTreesRegressor(n_estimators=100, max_depth=10, random_state=2021)

estimators= [('rf_reg',RandomForestRegressor(max_depth=10, n_jobs=-1, n_estimators=500, random_state=2021)),
('aba_reg',AdaBoostRegressor(n_estimators=500,learning_rate=0.3,random_state=2021,loss='square')),
('dt_reg',DecisionTreeRegressor(max_depth=10,random_state=2021))]

reg= StackingRegressor(estimators=estimators, final_estimator=best_reg)
reg.fit(X_train, y_train)